In [2]:
import os
import requests
import pandas as pd
from datetime import datetime

KRAKEN_BASE_API = "https://futures.kraken.com/derivatives/api/v3"

In [4]:
def get_instruments(symbol: str) -> dict:
    """
    @param symbol, supports [btc (xbt), eth, ltc, bch, xrp]    
    @returns instruments (sorted by expiration date)
    """
    
    # symbol check
    symbol = symbol.lower()
    valid_symbols = ['btc', 'eth', 'ltc', 'bch', 'xrp']
    if symbol not in valid_symbols:
        raise ValueError(f'< {symbol} > is not valid, must be in {valid_symbols}')
        
    if symbol == 'btc':
        symbol = 'xbt'

        
    # call instruments API   
    resp = requests.get(os.path.join(KRAKEN_BASE_API, "instruments"))
    j = resp.json()
    
    instruments = {}
    
    for d in j['instruments']:
        
        # check for inverse perp and for flexible perp
        is_future_inverse = \
            ('underlying' in d) and \
            (d['underlying'] == f'rr_{symbol}usd') and \
            (d['type'] == 'futures_inverse')

        is_future_flexible = \
            ('symbol' in d) and \
            (d['symbol'] == f'PF_{symbol.upper()}USD') and \
            (d['type'] == 'flexible_futures')
    
        if is_future_inverse or is_future_flexible:

            last_trading_time = d.get('lastTradingTime')
            last_trading_time_ms = None
            # convert string to milliseconds
            if last_trading_time:
                last_trading_time_ms = int(datetime.strptime(
                    last_trading_time, 
                    '%Y-%m-%dT%H:%M:%S.%fZ').timestamp() * 1000
                )

            instruments[d['symbol']] = {
                'type': d['type'],
                'underlying': d.get('underlying'),
                # 'base_ccy': symbol_original.upper(),
                # 'last_trading_time': last_trading_time,
                'expiration_timestamp_ms': last_trading_time_ms,
                'type': d['type'],
                'funding_rate_coefficient': d.get('fundingRateCoefficient'),
            }
            
    return instruments


def get_tickers(symbols: str) -> dict:
    """
    @param symbols list of symbols to return prices for
    @return close price of most resolution
    """
    
    resp = requests.get(os.path.join(KRAKEN_BASE_API, "tickers"))
    j = resp.json()
    tickers = j['tickers']
    prices = {d['symbol']: d['markPrice'] for d in tickers if d['symbol'].upper() in symbols}
    return prices


def get_last_funding(symbol: str):
    """
    @notice return last funding payment
    
    @return last funding payment $, last funding timestamp
    """
    resp = requests.get(f"https://futures.kraken.com/derivatives/api/v3/v4/historicalfundingrates/{symbol}")
    j = resp.json()
    return j

In [21]:
instruments = get_instruments('ltc')
marks = get_tickers(instruments)
instruments_df = pd.DataFrame(instruments).T
# add explicit sort by timestamp
instruments_df.sort_values(by='expiration_timestamp_ms', na_position='first', inplace=True)
# add mark prices
instruments_df['mark_price'] = pd.Series(marks)
# rename index to 'instrument_name'
# (after adding mark prices - so index match)
instruments_df.reset_index(inplace=True)
instruments_df.rename(columns={'index': 'instrument_name'}, inplace=True)

display(instruments_df)

,instrument_name,type,underlying,expiration_timestamp_ms,funding_rate_coefficient,mark_price
0,PI_LTCUSD,futures_inverse,rr_ltcusd,None,24,78.63
1,PF_LTCUSD,flexible_futures,None,None,24,78.64
2,FI_LTCUSD_240531,futures_inverse,rr_ltcusd,1717164000000,None,78.85
3,FI_LTCUSD_240628,futures_inverse,rr_ltcusd,1719583200000,None,79.50


In [8]:
# flexible futures are multi-asset perp?
# add last perp funding payment

In [ ]:
# instrument_name   base_ccy    expiration_timestamp_ms    mark_price      last_funding
# YES               YES        YES                         YES             NO